In [22]:
import os
import dotenv
from pathlib import Path

In [43]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import RetrievalQA


dotenv.load_dotenv()

True

In [17]:
doc_paths = [
    "../app/assets/docs/test_rag.txt",
    "../app/assets/docs/TM 2023 TCFD Report (Final version).docx",
]

In [ ]:
# cannot read tables and image
# To read tables, you need lots of gymnastics like opencv, tesseract, ocr, etc.
# Better off paying for unstructured API that has a very painful setup and also costs $1-10 per 1000 pages
# For simple use case, just copy paste tables in a separate txt file or use another LLM to extract tables


docs = [] 
for file in doc_paths:
    fpath = Path(file)

    try:
        if file.endswith(".pdf"):
            loader = PyPDFLoader(fpath)
        elif file.endswith(".docx"):
            loader = Docx2txtLoader(fpath)
        elif file.endswith(".txt") or file.endswith(".md"):
            loader = TextLoader(fpath)
        else:
            print(f"Unsupported file type: {file.type}")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading file: {file}")

In [23]:
# Load URLs

url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")

In [25]:
docs

[Document(metadata={'source': '..\\app\\assets\\docs\\test_rag.txt'}, page_content='The People Also Ask (PAA) Answer Generator is a tool that uses the OpenAI GPT-4o model and a SERP scraper to generate SEO optimized answers for the popular "People Also Ask" boxes on Google Search Result Pages (SERPs). It allows you to enter a location and keyword, then retrieves the current People Also Ask (PAA) box for that keyword and generates improved answers.'),
 Document(metadata={'source': '..\\app\\assets\\docs\\TM 2023 TCFD Report (Final version).docx'}, page_content='DRAFT\n\nDRAFT\n\n\n\nDRAFT\n\nDRAFT\n\n\n\n\n\n\n\n\n\n\n\n[Report to adopt approved TM IAR 2023 design]\n\n[Report to adopt approved TM IAR 2023 design]\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTelekom Malaysia Berhad (TM)\n\nDRAFT v5.5| Task Force on Climate-related Financial Disclosures (TCFD) Report 2023\n\n\n\nApril 2024\n\n\nTable of Contents\n\n\t\n\n\n\nTable of Contents\t2\n\nAbbreviations\t3\n\n1. Ove

In [28]:
# Split docs
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)
document_chunks = text_splitter.split_documents(docs)

In [30]:
document_chunks[2]

Document(metadata={'source': '..\\app\\assets\\docs\\TM 2023 TCFD Report (Final version).docx'}, page_content='MWh\n\nMegawatt-hour\n\nNGFS\n\nNetwork for Greening the Financial System\n\nNZE\n\nNet Zero Emissions\n\nRCP\n\nRepresentative Concentration Pathway\n\nSSP\n\nShared Socioeconomic Pathways\n\nSWG\n\nSustainability Working Group\n\nTCFD\n\nTask Force on Climate-related Financial Disclosures\n\ntCO2e\n\nTonnes of carbon emissions equivalent\n\nTM\n\nTelekom Malaysia Berhad\n\nTNB\n\nTenaga Nasional Berhad\n\nUSD\n\nUnited States Dollar\n\nWEO\n\nWorld Energy Outlook\n\n\n\n\n\n\nOverview\n\n\t\n\n1.1 A message from our Group Chief Executive Officer\n\n\n\nClimate change poses a growing threat to society and economic sustainability with an urgent need to adapt and mitigate its impact. Currently, 196 countries have committed towards becoming net-zero emission nations, to address climate change while creating a transparent monitoring and reporting framework. As a party to the Pari

In [32]:
vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

In [41]:
retriever = vector_db.as_retriever()

In [42]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000027E53AB2E90>, search_kwargs={})

In [44]:
# Augmented Generation
llm_stream_openai = ChatOpenAI(
    model="gpt-4o-mini",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.3,
    streaming=True,
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-haiku-latest",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai  # Select between OpenAI and Anthropic models for the response

In [45]:
conversation_rag_chain = RetrievalQA.from_chain_type(
    llm=llm_stream,  # Language Model (OpenAI GPT or Anthropic Claude)
    retriever=retriever,  # Retriever created from the vector DB
    return_source_documents=True,  # Optionally include the retrieved context in the output
)

In [34]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [35]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [ ]:

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]


In [38]:
messages

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi there! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the latest version of Streamlit?', additional_kwargs={}, response_metadata={}),
 {'role': 'assistant',
  'content': '*(RAG Response)*\nThe latest version of Streamlit is 1.40.0, which was released on November 6, 2024. If you need more information about its features or updates, feel free to ask!'}]

In [ ]:
conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"

for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

The latest version of Streamlit is 1.40.0, which was released on November 6, 2024. If you need more information about its features or updates, feel free to ask!

In [39]:
conversation_rag_chain.stream(
    input='What is the capital of France?'
)

<generator object RunnableBindingBase.stream at 0x0000027E534CF970>

In [40]:
conversation_rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000027E53AB2E90>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['input', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annot